In [1]:


from commonfunctions import *
inpath = '../inputs/'
outpath = '../outputs/segmentation/'
from scipy.signal import find_peaks, find_peaks_cwt
import time
import joblib

# load model
filename = '../music_notes_NN.joblib.pkl'
nn = joblib.load(filename)
classes = nn.classes_

line_to_char = {}
line_to_char[13] = 'c1'
line_to_char[12] = 'd1'
line_to_char[11] = 'e1'
line_to_char[10] = 'f1'
line_to_char[9] = 'g1'
line_to_char[8] = 'a1'
line_to_char[7] = 'b1'
line_to_char[6] = 'c2'
line_to_char[5] = 'd2'
line_to_char[4] = 'e2'
line_to_char[3] = 'f2'
line_to_char[2] = 'g2'
line_to_char[1] = 'a2'
line_to_char[0] = 'b2'

In [2]:
def separateStaffs(img, thickness, distance):
    rows_sum = np.sum(1 - img, axis=1)
    lines,_  = find_peaks(rows_sum, height = img.shape[1] * 0.5, distance=distance+thickness//2)
    parts = [0]

    for i in range(len(lines) - 1):
        if lines[i + 1] - lines[i] > 1.5 * distance:
            parts.append((lines[i + 1] + lines[i]) // 2)

    parts.append(img.shape[0]-1)

    return parts
    
def getStaffs(image):
    h, w = image.shape
    window , width = get_dimensions(w,h)
    hist = histogram(image,nbins=256,source_range='dtype')
    hist_acc = np.cumsum(hist[0])
    tratio = ((hist_acc[255]-hist_acc[206] + hist_acc[49]-hist_acc[0]) / hist_acc[-1])*100
    segmented = []
    if tratio < 4:
        segmented = segement_hand_written(image)
    elif 75 > tratio > 4:
        segmented = binraization(image,window,35)
    else: 
        segmented = (image > 150).astype("uint8")

    thickness, distance = get_lengthes(segmented)
    fimg, candidates = get_candidates_lines(segmented, thickness)
    fimg = filter_candidates_lines(fimg,candidates,thickness,distance)

    parts = separateStaffs(fimg, thickness, distance)
    staffs = []
    for start, end in zip(parts[:-1],parts[1:]):
        staffs.append(segmented[start:end,:])
    return staffs

In [3]:
target_img_size = (32, 32) 
def extract_hog_features(img):
    """
    TODO
    You won't implement anything in this function. You just need to understand it 
    and understand its parameters (i.e win_size, cell_size, ... etc)
    """
    img = cv2.resize(img, target_img_size)
    win_size = (32, 32)
    cell_size = (4, 4)
    block_size_in_cells = (2, 2)
    
    block_size = (block_size_in_cells[1] * cell_size[1], block_size_in_cells[0] * cell_size[0])
    block_stride = (cell_size[1], cell_size[0])
    nbins = 9  # Number of orientation bins
    hog = cv2.HOGDescriptor(win_size, block_size, block_stride, cell_size, nbins)
    h = hog.compute(img)
    h = h.flatten()
    return h.flatten()



def getDots(img_isolated):
    contours, _ = cv2.findContours(img_isolated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    dots = []
    for contour in contours:
        x,y,w,h = cv2.boundingRect(contour)
        if ( d//4 < w < d//2 and d//4 < h < d//2) :
            deltax = w//4
            deltay = h//4  
            sum = np.sum(img_isolated[y+deltay:y+h-deltay,x+deltax:x+w-deltax]//255)
            if sum/((h-2*deltay)*(w-2*deltax)) > 0.9:
                dots.append((x,y,w,h))
    return dots

def getContours(img):
    kernel = np.array([
        [0, 1, 0],
        [0, 1, 0],
        [0, 1, 0]
    ], np.uint8)
    img = cv2.dilate(img,kernel,iterations = 10) # to merge 2 number together
    contours, _ = cv2.findContours(img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    boxes = []
    cdots = []
    dots  = []
    for contour in contours:
        x,y,w,h = cv2.boundingRect(contour)
        if w > d//2 and h > d:
            i = len(boxes) - 1
            if i == -1 or not (x + w <= boxes[i][0] + boxes[i][2] and x >= boxes[i][0]): # remove overlaped contours
                sum = np.sum((img[y:y+h,x:x+w]//255))
                if(sum/(h*w) < .95):
                    boxes.append((x, y, w, h))
    return np.array(boxes) 

def xProjection(img, r):
    "Return a list containing the sum of the pixels in each column"
    (h, w) = img.shape[:2]
    sumCols = []
    for j in range(0, w, r):
        mx = 0
        tmp = 0
        for i in range(h):
            if np.all(img[i, j : j + r] == 255):
                tmp += 1
            else:
                mx = max(mx, tmp)
                tmp = 0
        mx = max(mx, tmp)    
        sumCols.append(mx)
    return np.array(sumCols)

def yProjection(img, r):
    "Return a list containing the sum of the pixels in each row"
    (h, w) = img.shape[:2]
    sumRows = []
    for i in range(0, h, r):
        tmp = 0
        for j in range(w):
            if img[i : i + r, j] == 255:
                tmp += 1
        sumRows.append(tmp)
    return np.array(sumRows)

def getBeamHeads(img, d):
    x = xProjection(img, 8)
    x[x < d // 2] = 0
    
    points = []
    tx1, tx2 = -1, -1
    for i in range(len(x)):
        if x[i]:
            if tx1 == -1:
                tx1 = i * 8

        elif tx1 != -1:
            tx2 = i * 8
            points.append((tx1, tx2))
            tx1, tx2 = -1, -1
            
    if tx1 != -1:
        tx2 = (len(x) - 1) * 8
        points.append((tx1, tx2))
        
    mask = img.copy()
    mask[:] = 0

    (h, w) = img.shape[:2]
    for px1, px2 in points:
        cv2.rectangle(mask,(px1,0),(px2,h),(255,255,255),-1)
    
    img[mask != 255] = 0
    kernel = np.ones((3,3), np.uint8)
    img = cv2.erode(img,kernel,iterations = 3)

    contours, _ = cv2.findContours(img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    boxes = []
    for contour in contours:
        x,y,w,h = cv2.boundingRect(contour)
        if h > d // 2:
            i = len(boxes) - 1
            if i == -1 or not (x + w <= boxes[i][0] + boxes[i][2] and x >= boxes[i][0]): # remove overlaped contours
                boxes.append((x, y, w, h))
                
                    
    return np.array(boxes)

def getNoteHeads(img, d):
    y = yProjection(img, 1)
    th = np.max(y) - 5
    y[y < th] = 0

    points = []
    ty1, ty2 = -1, -1
    for i in range(len(y)):
        if y[i]:
            if ty1 == -1:
                ty1 = i

        elif ty1 != -1:
            ty2 = i
            points.append((ty1, ty2))
            ty1, ty2 = -1, -1
            
    if ty1 != -1:
        ty2 = (len(y) - 1)
        points.append((ty1, ty2))

    boxes = []
    (h, w) = img.shape[:2]
    
    if len(points):
        points = sorted(points)
        y1, y2 = points[0]
        boxes.append((0,y1,w,y2-y1))
    
    return np.array(boxes)

def getChordHeads(img, d):
    kernel = np.ones((3,3), np.uint8)
    img = cv2.erode(img,kernel,iterations = 3)    
    y = yProjection(img, 1)
    y[y < d//2] = 0

    points = []
    ty1, ty2 = -1, -1
    for i in range(len(y)):
        if y[i]:
            if ty1 == -1:
                ty1 = i

        elif ty1 != -1:
            ty2 = i
            points.append((ty1, ty2))
            ty1, ty2 = -1, -1
            
    if ty1 != -1:
        ty2 = (len(y) - 1)
        points.append((ty1, ty2))

    boxes = []
    (h, w) = img.shape[:2]
    
    if len(points):
        points = sorted(points)
        for p in points:
            y1, y2 = p
            boxes.append((0,y1,w,y2-y1))
    
    return np.array(boxes)

def getLines(img):
    img *= 255
    img = 255 - img
    y =  yProjection(img, 1)
    y[y < 100] = 0
        
    points = []
    ty1, ty2 = -1, -1
    for i in range(len(y)):
        if y[i]:
            if ty1 == -1:
                ty1 = i

        elif ty1 != -1:
            ty2 = i
            points.append((ty1, ty2))
            ty1, ty2 = -1, -1

    if ty1 != -1:
        ty2 = (len(y) - 1)
        points.append((ty1, ty2))
    
    points = np.array(points)
    points = [(p[0] + p[1]) // 2 for p in points]
        
    d = 0
    for i in range(1, len(points)):
        d += abs(points[i - 1] - points[i])
    d //= len(points) - 1
    
    p1 = points[0]
    for i in range(1):
        tmp = p1 - (i + 1) * d
        points.insert(0, tmp)
    p1 = points[-1]
    for i in range(1):
        tmp = p1 + (i + 1) * d
        points.append(tmp)
        
    tmp_points = []
    
    for p in points:
        tmp_points.append(p)
        tmp_points.append(p + (d // 2))
    
    tmp_points.pop(len(tmp_points) - 1)
    p1 = tmp_points[0] - (d // 2)
    tmp_points.insert(0, p1)
    
    return tmp_points, d

def notHalf(img, d):
    x = xProjection(img, 1)
    x[x < d//2] = 0
    x[x > 2*d] = 0
    peak = np.max(x)
    return peak >= d

In [4]:
 def get_candidates_lines(bimg, thickness):
    fimg = np.copy(bimg)
    candidates = [] # Contains list of candidate staffs lines (row, begin, height)
    cols = fimg.shape[1]
    rows = fimg.shape[0]
    delta = max(1 , thickness//3)
    t2 = thickness+ delta
    t1 = abs(thickness-delta)
    flag = False
    for i in range(cols):
        for j in range(rows):
            if fimg[j,i] == 0 and flag == False:
                beg = j
                flag = True
            elif fimg[j,i] == 1 and flag == True:
                flag = False
                if j-beg > t2 or j-beg < t1:
                    fimg[beg:j, i] = 1
                else:
                    candidates.append((i, beg, j-beg))
    return fimg, candidates


In [5]:
def filter_candidates_lines(bimg,v,thickness , distance):
    fimg = np.copy(bimg)
    candidates = np.copy(v) 
    cols = fimg.shape[1]
    rows = fimg.shape[0]
    for c in candidates:
        i, beg, t = c
        if np.any(fimg[beg-distance+thickness:beg-1, i] == 0):
            fimg[beg:beg+t, i] = 1
    return fimg


In [6]:
def binraization(img,n=8,t=15):

    outputimg = np.zeros(img.shape)
    intimg = np.zeros(img.shape)
    h = img.shape[1]
    w = img.shape[0]
    s= min(w,h)//n
    count = s**2
    img = np.pad(img,s,"constant")
    intimg = np.cumsum(img ,axis =1)
    intimg = np.cumsum(intimg ,axis =0)
    a = np.roll(intimg,-s//2,axis =0)
    a = np.roll(a,-s//2,axis =1)
    a[:,-s//2:]=a[-s//2-1,-s//2-1]
    a[-s//2:,:]=a[-s//2-1,-s//2-1]
    b = np.roll(intimg,s//2+1,axis =0)
    b = np.roll(b,-s//2,axis =1)
    b[0:s//2+1,:]=0
    b[:,-s//2:]=0
    
    c = np.roll(intimg,s//2+1,axis =1)
    c = np.roll(c,-s//2,axis =0)
    c[:,0:s//2+1]=0
    c[-s//2:,:]=0
    
    d = np.roll(intimg,s//2+1,axis =0)
    d = np.roll(d,s//2+1,axis =1)
    d[0:s//2+1,:]=0
    d[:,0:s//2+1]=0

    sum = (a-b-c+d)*(100-t)/100
    outputimg = np.where(img>sum/count, 1, 0)
    return outputimg[s:-s,s:-s]

In [7]:
def segement_hand_written(img):
    h , w = img.shape 
    window , width = get_dimensions(w,h)
    fimg = median(img)
    glares = (fimg > 195)
    bimg = binraization(fimg,window,7)
    output = (bimg ) - (glares)
    output = median(output)
    output = binary_dilation(1-output)
    vse = np.array([[0,1,0],
                    [0,1,0],
                    [0,1,0]      ])

    hse = np.array([[0,0,0],
                    [1,1,1],
                    [0,0,0]
    ])
    output = binary_dilation(output , vse)
    output = binary_dilation(output , hse)
    output = 1 - output
    return output #the reuslt is binary image

In [8]:
def remove_staff_lines(bimg , t , d,hw):
    csl , c = get_candidates_lines(bimg,t) 
    fcsl  = filter_candidates_lines(csl,c,t,d)
    staffless = (bimg-fcsl)
    if(hw):
        staffless = median(staffless)
        staffless = binary_closing(staffless,np.ones((d//2,d//2)))
        staffless = median(staffless)
    result = ((staffless)).astype("uint8")
    return result  #the result is a uint8 bw image

In [9]:
tcimgs , tcfiles = load_images_from_folder(inpath + "test-set-camera-captured/test-cases")
hwimgs , hwfiles = load_images_from_folder(inpath+"test-set-camera-captured/test-cases")
cimgs , cfiles = load_images_from_folder(inpath+"test-set-camera-captured/test-cases")
simgs , sfiles = load_images_from_folder(inpath+"test-set-scanned/test-cases")

../inputs/test-set-camera-captured/test-cases
../inputs/test-set-camera-captured/test-cases
../inputs/test-set-camera-captured/test-cases
../inputs/test-set-scanned/test-cases


In [10]:

#the working cell to get contours
path = outpath
files = tcfiles
images = tcimgs
start_time = time.time()
for i , img in enumerate(images) :
    cimg = cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)
    h , w = img.shape 
    window , width = get_dimensions(w,h)
    hist = histogram(img,nbins=256,source_range='dtype')
    hist_acc =np.cumsum(hist[0])
    tratio = ((hist_acc[255]-hist_acc[206] + hist_acc[49]-hist_acc[0]) / hist_acc[-1])*100
  
    if ( tratio < 4):
        bimg = segement_hand_written(img)
    elif ( 75 > tratio > 4):
        bimg = binraization(img,window,35)
    else: 
        bimg = (img > 150).astype("uint8")
        
    bimg2 = bimg.copy()
    t,d = get_lengthes(bimg)
    csl , c = get_candidates_lines(bimg2,t) 
    fcsl ,c = filter_candidates_lines(csl,c,t,d)
    lines, d2 = getLines(csl)
    #print(lines)
    img_isolated = remove_staff_lines(bimg,t,d , tratio < 4 )
    img_isolated = img_isolated.astype("uint8")
    show_images([img_isolated])
    # get dots
    contours, _ = cv2.findContours(img_isolated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    temp_img = img.copy()
    backtorgb = cv2.cvtColor(temp_img,cv2.COLOR_GRAY2RGB)
    cdots = []
    dots = []
    for contour in contours:
        x,y,w,h = cv2.boundingRect(contour)
        if(w > d//2 and h > d):
            cv2.rectangle(backtorgb,(x,y),(x+w,y+h),(50,100,200),3)
        elif ( d//4 < w < d//2 and d//4 < h < d//2) :
            cdots.append((x,y,w,h))
    for dot in cdots:
        x,y,w,h = dot
        deltax = w//4
        deltay = h//4  
        sum = np.sum(img_isolated[y+deltay:y+h-deltay,x+deltax:x+w-deltax]//255)
        if sum/((h-2*deltay)*(w-2*deltax)) > 0.9:
            cv2.rectangle(backtorgb,(x,y),(x+w,y+h),(220,50,50),3)
            dots.append(dot)
    
    boxes = getContours(img_isolated) # now we have contours after fixing
    if len(dots):
        dots = np.array(dots)
        boxes = np.concatenate((boxes, dots), axis=0)
    boxes = sorted(boxes, key=lambda t: t[0])
    notes = []
    accidentals = "" # Accidentals # &  
    for box in boxes:
        x,y,w,h = box
        cv2.rectangle(cimg,(x,y),(x+w,y+h),(50,100,200),3)
        croped = img_isolated[y:y+h, x:x+w]
        features = extract_hog_features(croped)
        note_type = str(classes[np.argmax( nn.predict_proba([features]))])
        
        # fix wrong dots
        if note_type == "dot" and box not in dots:
            note_type = 'a_1'
        
        if (len(note_type) >= 3 and note_type[1] == '_'):
            f = note_type.split('_')[1]
            if f == '2' and notHalf(croped, d):
                f = '4'
           
            heads = None
            if note_type[0] == 'a':
                heads = getNoteHeads(croped, d2) # heads boxes
            elif note_type[0] == 'b':
                heads = getBeamHeads(croped, d2)
            
            if heads is None:
                continue
            
            for head in heads:
                xh,yh,wh,hh = head
                yh = yh + (hh // 2) + y
                xh = xh + x
                #print(yh)
                idx = np.abs(lines - yh).argmin()
                char = line_to_char[idx]
                char = char[:1] + accidentals + char[1:]
                accidentals = ""
                notes.append(char + '/' + f)
                cv2.rectangle(cimg, (xh,yh), (xh+wh,yh+hh), (255,0,0),3)
        elif note_type == "sharp":
            accidentals = "#"
        elif note_type == "flat":
            accidentals = "&"
        elif note_type == "double_sharp":
            accidentals = "##"
        elif note_type == "double_flat":
            accidentals = "&&"
        elif note_type == "dot" and len(notes):
            notes[-1] += '.'
        elif note_type == "chord":
            heads = getChordHeads(croped, d2)
            
            if heads is None:
                continue
            note = "{"
            for head in heads:
                if note[-1] != "{":
                    note += ","
                    
                xh,yh,wh,hh = head
                yh = yh + (hh // 2) + y
                xh = xh + x
                #print(yh)
                idx = np.abs(lines - yh).argmin()
                char = line_to_char[idx]
                char = char[:1] + accidentals + char[1:]
                accidentals = ""
                note += (char + '/' + '4')
                cv2.rectangle(cimg, (xh,yh), (xh+wh,yh+hh), (255,0,0),3)
            note += "}"
            notes.append(note)
        
    show_images([cimg])
    print(notes)
print("--- %s seconds ---" % (time.time() - start_time))
    

ValueError: too many values to unpack (expected 2)

In [27]:
    img = cv2.imread(os.path.join(inpath + "test-set-scanned/test-cases", "test6.png"))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    staffs = getStaffs(img)
    sol = []
    for bimg in staffs:
        cimg = cv2.cvtColor((bimg*255).astype("uint8"),cv2.COLOR_GRAY2RGB)
        bimg2 = bimg.copy()
        t,d = get_lengthes(bimg)
        csl , c = get_candidates_lines(bimg2,t) 
        fcsl  = filter_candidates_lines(csl,c,t,d)
        img_isolated = remove_staff_lines(bimg,t,d , False)
        lines, d2 = getLines(fcsl)
        # get dots
        dots = getDots(img_isolated)
        boxes = getContours(img_isolated) # now we have contours after fixing
        
        if len(dots):
            dots = np.array(dots)
            boxes = np.concatenate((boxes, dots), axis=0)
        boxes = sorted(boxes, key=lambda t: t[0])
        notes = []
        accidentals = "" # Accidentals # &  
        for box in boxes:
            x,y,w,h = box
            cv2.rectangle(cimg,(x,y),(x+w,y+h),(50,100,200),3)
        show_images([cimg])
        for box in boxes:
            x,y,w,h = box
            cv2.rectangle(cimg,(x,y),(x+w,y+h),(50,100,200),3)
            croped = img_isolated[y:y+h, x:x+w]
            #show_images([croped])
            features = extract_hog_features(croped)
            note_type = str(classes[np.argmax( nn.predict_proba([features]))])
            
            # fix wrong dots
            if note_type == "dot" and box not in dots:
                note_type = 'a_1'
            
            if (len(note_type) >= 3 and note_type[1] == '_'):
                f = note_type.split('_')[1]
                if f == '2' and notHalf(croped, d):
                    f = '4'
            
                heads = None
                if note_type[0] == 'a':
                    heads = getNoteHeads(croped, d2) # heads boxes
                elif note_type[0] == 'b':
                    heads = getBeamHeads(croped, d2)
                
                if heads is None:
                    continue
                
                for head in heads:
                    xh,yh,wh,hh = head
                    yh = yh + (hh // 2) + y
                    xh = xh + x
                    #print(yh)
                    idx = np.abs(lines - yh).argmin()
                    char = line_to_char[idx]
                    char = char[:1] + accidentals + char[1:]
                    accidentals = ""
                    notes.append(char + '/' + f)
                    cv2.rectangle(cimg, (xh,yh), (xh+wh,yh+hh), (255,0,0),3)
            elif note_type == "sharp":
                accidentals = "#"
            elif note_type == "flat":
                accidentals = "&"
            elif note_type == "double_sharp":
                accidentals = "##"
            elif note_type == "double_flat":
                accidentals = "&&"
            elif note_type == "dot" and len(notes):
                notes[-1] += '.'
            elif note_type == "chord":
                heads = getChordHeads(croped, d2)
                
                if heads is None:
                    continue
                note = "{"
                for head in heads:
                    if note[-1] != "{":
                        note += ","
                        
                    xh,yh,wh,hh = head
                    yh = yh + (hh // 2) + y
                    xh = xh + x
                    #print(yh)
                    idx = np.abs(lines - yh).argmin()
                    char = line_to_char[idx]
                    char = char[:1] + accidentals + char[1:]
                    accidentals = ""
                    note += (char + '/' + '4')
                    cv2.rectangle(cimg, (xh,yh), (xh+wh,yh+hh), (255,0,0),3)
                note += "}"
                notes.append(note)
            
        show_images([cimg])
        sol.append(" ".join(notes))
        
    numberOfSols = len(sol)
    sol = ",\n".join(map("[ {} ]".format, sol))
    with open("out.txt", "w") as f:
        if numberOfSols == 1:
            f.write(sol)
        elif numberOfSols > 1:
            f.write("{{\n{}\n}}".format(sol))
    print(sol)

            

IndexError: list index out of range